Code to import revenue data from Brazilian governments
Source: "https://www.gov.br/receitafederal/pt-br/centrais-de-conteudo/publicacoes/relatorios/arrecadacao-federal"

In [1]:
# Importing the required libraries
import requests
from bs4 import BeautifulSoup
import os
import PyPDF2
import re
import csv
import tabula

In [126]:
# Define the target url
url = "https://www.gov.br/receitafederal/pt-br/centrais-de-conteudo/publicacoes/relatorios/arrecadacao-federal"

# Check if the target url is reachable
response = requests.get(url)
# Define the parser for the response
soup = BeautifulSoup(response.text, "html.parser")
# Check the url to get all yearly links
year_links = soup.find_all(lambda tag: tag.name == "a" and tag.get("class") == ["govbr-card-content"] and tag.get("href") is not None)
# subset yearly links to get only data from 2023 to 2001
year_links = year_links[1:24]




In [ ]:
# Let's get the links for the monthly reports
report_links = []
# Loop through the yearly links
for year_link in year_links:
    year = year_link.text.strip()  #Get the year
    year_url = year_link["href"]   #Get the url

    # Check if the target url is reachable
    response_year = requests.get(year_url)
    soup_year = BeautifulSoup(response_year.text, "html.parser")

    # Get the links for the monthly reports in the first page
    report_links += soup_year.find_all(lambda tag: tag.name == "a" and tag.get("class") == ["state-missing-value"] and tag.get("href") is not None)
    #Check if there is a second page
    try:
        second_page_check = soup_year.find('ul', class_='paginacao listingBar').find('a', text='2')
        if second_page_check: # if there is a second page
            second_page_url = second_page_check['href'] #get the url
            response_year = requests.get(second_page_url) #get the response
            soup_year = BeautifulSoup(response_year.text, "html.parser") #parse the response
            report_links += soup_year.find_all(lambda tag: tag.name == "a" and tag.get("class") == ["state-missing-value"] and tag.get("href") is not None) #get the links
    except:
        pass
    
    # Create a directory for each year
    output_dir = f"../data/downloads_revenue/{year}"
    os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Define a function to download the PDFs or excel files for each link
def download_pdf(link):
    # Check if the link is reachable
    response = requests.get(link)
    # Define the parser for the response
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the link for the PDF or excel file
    pdf_link = soup.find("div", id="content-core").find("a")["href"]
    # Get the name of the PDF or excel file
    pdf_name = soup.find("div", id="content-core").find("a").text.strip()
    # Split the URL by "/" to get a list of its components and find the index of "arrecadacao-federal" in the list
    arrecadacao_federal_index = link.split("/").index("arrecadacao-federal")
    # Get the year from the next component after "arrecadacao-federal"
    pdf_year = link.split("/")[arrecadacao_federal_index + 1]
    # Define the file path
    output_dir = f"../data/downloads/{pdf_year}/"
    # create the complete file name
    pdf_path = os.path.join(output_dir, pdf_name)
    # Check if the file already exists
    if not os.path.exists(pdf_path):
        with open(pdf_path, "wb") as f:
            #Download the PDF and save it to the file
            f.write(requests.get(pdf_link).content)


# Itera sobre os links dos relatorios
for report_link in report_links:
    month_url = report_link["href"]    # Get the url for the month
    download_pdf(month_url)